In [15]:
%reload_ext autoreload
%autoreload 2

import os
fdel = os.path.sep
import sys
wd = %pwd
sys.path.append(wd)
import altair as alt
import pandas as pd
print(wd)

# data = etf.btc_etf_data(metric = "btc_etf_aum", export_response = True)
# data.df
#data_2 = etf.btc_etf_data(metric = "btc_etf_aum_hist", export_response = True)
# data = etf.json_file_io(filename= wd + fdel + "backend"+fdel+"btc_etf_aum_hist_theblock.json")
# data["chart"]["jsonFile"]["Series"]

/Users/jamesbishop/Documents/Python/New Endeavors/BTC_ETF_Tracker/backend


In [ ]:
from bs4 import BeautifulSoup
import requests

def get_html_save(url: str, save_path: str = wd+fdel+'last_request.html', save: bool = True):
    r = requests.get(url)
    if save:
        with open(save_path, 'w') as wp:
            wp.write(r.text)
    return r.text  

#html = get_html_save("https://farside.co.uk/?p=997", save=True)
with open(wd+fdel+'last_request.html', 'r') as wp:
    html = wp.read()
soup = BeautifulSoup(html, features="html.parser"); dumpstr = ""

tag = soup.table 
for span in tag.find_all('span'):
    dumpstr+=span.get_text().strip()+"\n"
liststr = dumpstr.split("\n")  
dropafter = liststr.index("Average")
liststr = liststr[:dropafter]
index = []; vals = []; colnames = []; notmatched = []; totalcount = 0
print(liststr)

In [ ]:

# data_melted = data.reset_index(drop=False).melt('Timestamp', var_name='Fund', value_name='Net Flow (USD)')
# data_melted
# Create a bar chart for each column and layer them on top of each other
data_melted = aum_data.reset_index().melt('Timestamp', var_name='Category', value_name='Values') 
data_melted
chart = alt.Chart(data_melted).mark_bar().encode(
    x=alt.X('Timestamp:T', axis=alt.Axis(title='Date')),
    y=alt.Y('Values:Q'),
    color='Category:N',
    # tooltip=['Category']
).interactive()
chart
# chart = alt.Chart(data).mark_bar().encode(x = 'Timestamp', y = 'GBTC').interactive()
# chart

alt.Chart(...)

In [17]:
import btc_etfs

aum_dollars = btc_etfs.btc_etf_data()(metric="etf_aum_daily", export_response=True)
aum_dollars
aum_dollars.resp
aum_dollars.url
aum_dollars.df
#aum_dollars.to_excel(wd+fdel+"hybrid_flows_table.xlsx")

Requesting data from theblock for etf_aum_daily..
200
Data successfully retrieved.
Dataset:  Spot Bitcoin ETF AUM (Daily) 
Start:  2024-01-11 00:00:00 
Updated last:  2024-05-25 05:27:33
JSON format data has been saved to the file:  /Users/jamesbishop/Documents/Python/New Endeavors/BTC_ETF_Tracker/backend/etf_aum_daily_theblock.json


/Users/jamesbishop/Documents/Python/New Endeavors/BTC_ETF_Tracker/backend/btc_etfs.py:146: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  i = 0; output_df = pd.Series()


,GBTC,IBIT,FBTC,ARKB,BITB,HODL,BTCO,BRRR,EZBC,BTCW
Timestamp,,,,,,,,,,
2024-01-11,2.866922e+10,1.058310e+07,2.030420e+07,3.831050e+07,2.531858e+06,7.619958e+07,5.108067e+06,2.685795e+07,2.693344e+06,3453105.45
2024-01-12,2.745090e+10,1.167646e+08,2.357309e+08,7.240437e+07,2.295719e+08,8.319861e+07,2.160992e+07,2.861267e+07,5.039344e+07,3313274.18
2024-01-13,2.638004e+10,1.122096e+08,2.265350e+08,6.957986e+07,2.206162e+08,7.995300e+07,2.076691e+07,2.749648e+07,4.842758e+07,3184022.27
2024-01-14,2.603674e+10,1.107493e+08,2.235870e+08,6.867438e+07,2.177452e+08,7.891254e+07,2.049666e+07,2.713866e+07,4.779737e+07,3142587.14
2024-01-15,2.596850e+10,1.104591e+08,2.230010e+08,6.849440e+07,2.171746e+08,7.870572e+07,2.044294e+07,2.706753e+07,4.767210e+07,3134350.90
...,...,...,...,...,...,...,...,...,...,...
2024-05-20,1.983471e+10,1.894843e+10,1.051326e+10,3.261110e+09,2.460581e+09,6.696434e+08,4.876554e+08,4.818510e+08,3.552966e+08,84070636.24
2024-05-21,2.040408e+10,1.948333e+10,1.081004e+10,3.422841e+09,2.558589e+09,6.825423e+08,5.014217e+08,4.954534e+08,3.653265e+08,87571436.24
2024-05-22,2.008237e+10,1.946646e+10,1.066549e+10,3.368873e+09,2.514071e+09,6.717807e+08,4.935158e+08,4.876416e+08,3.595664e+08,86190699.51


In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup("<p>Some<b>bad<i>HTML")
print(soup.prettify())

In [ ]:
import altair as alt 
import charts
hybrid_df = pd.read_excel(wd+fdel+"Hybrid_flowz_table.xlsx", index_col = 0)
fig = charts.altair_line(hybrid_df, right_columns = ['GBTC'])
fig